In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/face_data/train2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/face_data/train2


In [0]:
!pip install selectivesearch
import re
import cv2
import matplotlib.pyplot as plt
import selectivesearch
import scipy
import numpy as np
import matplotlib.patches as mpatches
import math
import glob
import time
import pdb
import pickle

In [0]:
def Find_Image_Points(filename):
    img_var = filename[:-4]
    f = open(str(img_var)+'.pts',"r")

    points = []

    for i,line in enumerate(f):
        if 2<i<71:
            points.append(line)
    x = []
    y = []

    for i in range(0,len(points)):
        points[i] = re.sub(r"\n","",points[i])
        points[i] = points[i].replace("'","")
        temp_x, temp_y = points[i].split()
        x.append(int(float(temp_x)))
        y.append(int(float(temp_y)))
    
    return x,y

In [0]:
def Scale_Image_And_Points(image,x,y):
    h_ratio = 227/image.shape[0]
    w_ratio = 227/image.shape[1]
    for i in range(0,len(x)):
        x[i] = int(x[i]*w_ratio)
        y[i] = int(y[i]*h_ratio)
    image = image/255
    image = scipy.misc.imresize(image, (227, 227))
    for i in range(0,len(x)):
        x[i] = x[i]/image.shape[1]
        y[i] = y[i]/image.shape[0]
    landmarks = y+x
    return image,landmarks

In [0]:
def Calculate_Pose_Estimation(image,x,y):
    im_size = image.shape
    image_points = np.array([
        (x[30], y[30]),
        (x[8], y[8]),
        (x[36], y[36]),
        (x[45],y[45]),
        (x[48], y[48]),
        (x[54],y[54])
    ], dtype = "double")

    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner

                        ])

    #Camera Internals
    focal_length = im_size[1]
    center = (im_size[1]/2, im_size[0]/2)
    camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )

    #print ("Camera Matrix :\n {0}".format(camera_matrix))

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)

    #print ("Rotation Vector:\n {0}".format(rotation_vector))
    #print ("Translation Vector:\n {0}".format(translation_vector))

    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    axis = np.float32([[500,0,0], 
                              [0,500,0], 
                              [0,0,500]])

    imgpts, jac = cv2.projectPoints(axis, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    modelpts, jac2 = cv2.projectPoints(model_points, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    rvec_matrix = cv2.Rodrigues(rotation_vector)[0]

    proj_matrix = np.hstack((rvec_matrix, translation_vector))
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6] 


    pitch, yaw, roll = [math.radians(_) for _ in eulerAngles]


    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))
    
    pose = np.array([roll,pitch,yaw])
    
    return pose

In [0]:
def selective_search(img, plot_image = False):
    img_lbl, regions = selectivesearch.selective_search(img, scale=200, sigma=0.9, min_size=100)
    candidates = set()
    for r in regions:
        x, y, w, h = r['rect']
        candidates.add(r['rect'])
    if plot_image:
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
        ax.imshow(img)
        for x, y, w, h in candidates:
            rect = mpatches.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=3)
            ax.add_patch(rect)
        plt.show()
    return candidates

In [0]:
def intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    if interArea == boxAArea & boxAArea != boxBArea:
        iou = 0.0
    else:
        iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
def get_xy(box):
    x,y,w,h = box
    x_top_left = x
    y_top_left = y
    x_right_bottom = x_top_left + w
    y_right_bottom = y_top_left + h
    return [x_top_left, y_top_left, x_right_bottom, y_right_bottom]

In [0]:
def region_of_proposals(image,x,y,pose):
    candidates = selective_search(image, False)   
    #selected_regions = list()
    #unselected_regions = list()
    boxA = [x[0]-20, y[24]-20,x[16]+10, y[8]+10]
    final_array = np.empty((1,4))
    for candidate in candidates:
        boxB = get_xy(candidate)
        iou = intersection_over_union(boxA, boxB)
        if iou >= 0.5:
            temp_x = []
            temp_y = []
            for i in range(0,len(x)):
                temp_x.append(x[i]-candidate[0])
                temp_y.append(y[i]-candidate[1])
            candidate_image = image[candidate[1]:candidate[1]+candidate[3], candidate[0]:candidate[0]+candidate[2],:]
            scaled_image,landmarks= Scale_Image_And_Points(candidate_image,temp_x,temp_y)
            face = np.array([1])
            temp_array = np.array([scaled_image,face,landmarks,pose])
            final_array = np.append(final_array,temp_array.reshape(1,temp_array.shape[0]),axis=0)
        elif 0.1<iou<= 0.11:
            temp_x = []
            temp_y = []
            for i in range(0,len(x)):
                temp_x.append(x[i]-candidate[0])
                temp_y.append(y[i]-candidate[1])
            candidate_image = image[candidate[1]:candidate[1]+candidate[3], candidate[0]:candidate[0]+candidate[2],:]
            scaled_image,landmarks = Scale_Image_And_Points(candidate_image,temp_x,temp_y)
            face_nil = np.array([0])
            #pose_nil = np.array([0,0,0])
            temp_array = np.array([scaled_image,face_nil,landmarks,pose])
            final_array = np.append(final_array,temp_array.reshape(1,temp_array.shape[0]),axis=0)
    # resize according to alexnet input and return selected regions
    #image = image / 255
    final_array = np.delete(final_array,(0),axis=0)
    return final_array

In [0]:
count_im = 0
dataset = np.empty((1,4))
face = np.array([1])
for filename in glob.glob('*.jpg'):
    count_im += 1
    #Opening the Points File for image=filename
    x,y = Find_Image_Points(filename)

    #Fetching the image for the filename
    image = cv2.imread(filename)
    
    #Calculating the roll,pictch, yaw for the image
    pose = Calculate_Pose_Estimation(image,x,y)
    
    a=max(0,y[24]-30)
    b=min(image.shape[0],y[8]+20)
    c=max(0, x[0]-30)
    d=min(image.shape[1], x[16]+30)
    boxA = image[a:b, c:d]
    
    for i in range(0,len(x)):
        x[i] = x[i]-c
        y[i] = y[i]-a
    scaled_image, landmarks = Scale_Image_And_Points( boxA, x, y)
    data=np.array([scaled_image, face, landmarks, pose])
    data=data.reshape(1,data.shape[0])
    dataset=np.append(dataset, data, axis=0)
    
    if dataset.shape[0]==100:
        dataset = np.delete(dataset,(0),axis=0)
        print("saving:",count_im)
        np.save(F"/content/gdrive/My Drive/face_data/npyfileslp/train2"+str(count_im)+".npy",dataset)
        del dataset
        dataset = np.empty((1,4))
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  


saving: 99
saving: 198
saving: 297
saving: 396


In [0]:
%cd /content/gdrive/My Drive/face_data/npyfileslp


/content/gdrive/My Drive/face_data/npyfileslp


In [0]:
%ls

helenlp1089.npy  helenlp198.npy  helenlphelentest198.npy  ifpwlp99.npy
helenlp1188.npy  helenlp297.npy  helenlphelentest297.npy  ifpwlptest198.npy
helenlp1287.npy  helenlp396.npy  helenlphelentest99.npy   ifpwlptest99.npy
helenlp1386.npy  helenlp495.npy  ifpwlp198.npy            train2198.npy
helenlp1485.npy  helenlp594.npy  ifpwlp297.npy            train2297.npy
helenlp1584.npy  helenlp693.npy  ifpwlp396.npy            train2396.npy
helenlp1683.npy  helenlp792.npy  ifpwlp495.npy            train299.npy
helenlp1782.npy  helenlp891.npy  ifpwlp594.npy
helenlp1881.npy  helenlp990.npy  ifpwlp693.npy
helenlp1980.npy  helenlp99.npy   ifpwlp792.npy


In [0]:
a1 = np.load('ifpwlptest99.npy' , allow_pickle=True)
a2 = np.load('ifpwlptest198.npy', allow_pickle=True)
a3 = np.load('train299.npy', allow_pickle=True)
a4 = np.load('ifpwlp693.npy', allow_pickle=True)
a5 = np.load('ifpwlp792.npy', allow_pickle=True)
a6 = np.load('train2198.npy', allow_pickle=True)
a7 = np.load('train2297.npy', allow_pickle=True)
a8 = np.load('train2396.npy', allow_pickle=True)
a9 = np.load('helenlp693.npy', allow_pickle=True)
a10 = np.load('helenlp1881.npy', allow_pickle=True)

In [0]:
a =np.concatenate((a1, a2, a3, a4, a5, a6, a7, a8, a9, a10), axis=0)

In [0]:
np.save('lp4000.npy',a)